# imports/setup

In [1]:
import datetime
import json
import pandas as pd
from collections import defaultdict
from typing import List, Dict, Any
from datamodel import TradingState, Listing, OrderDepth, Trade, Observation

# backtester

In [2]:
# edit to mark to mm mid 

In [3]:
class Backtester:
    def __init__(self, trader, listings: Dict[str, Listing], position_limit: Dict[str, int], fair_marks, 
                 market_data: pd.DataFrame, trade_history: pd.DataFrame, file_name: str = None):
        self.trader = trader
        self.listings = listings
        self.market_data = market_data
        self.position_limit = position_limit
        self.fair_marks = fair_marks
        self.trade_history = trade_history.sort_values(by=['timestamp', 'symbol'])
        self.file_name = file_name

        self.observations = [Observation({}, {}) for _ in range(len(market_data))]

        self.current_position = {product: 0 for product in self.listings.keys()}
        self.pnl_history = []
        self.pnl = {product: 0 for product in self.listings.keys()}
        self.cash = {product: 0 for product in self.listings.keys()}
        self.trades = []
        self.sandbox_logs = []
        
    def run(self):
        traderData = ""
        
        timestamp_group_md = self.market_data.groupby('timestamp')
        timestamp_group_th = self.trade_history.groupby('timestamp')
        
        own_trades = defaultdict(list)
        market_trades = defaultdict(list)
        pnl_product = defaultdict(float)
        
        trade_history_dict = {}
        
        for timestamp, group in timestamp_group_th:
            trades = []
            for _, row in group.iterrows():
                symbol = row['symbol']
                price = row['price']
                quantity = row['quantity']
                buyer = row['buyer'] if pd.notnull(row['buyer']) else ""
                seller = row['seller'] if pd.notnull(row['seller']) else ""

                
                trade = Trade(symbol, int(price), int(quantity), buyer, seller, timestamp)
                
                trades.append(trade)
            trade_history_dict[timestamp] = trades
        
        
        for timestamp, group in timestamp_group_md:
            order_depths = self._construct_order_depths(group)
            order_depths_matching = self._construct_order_depths(group)
            order_depths_pnl = self._construct_order_depths(group)
            state = self._construct_trading_state(traderData, timestamp, self.listings, order_depths, 
                                 dict(own_trades), dict(market_trades), self.current_position, self.observations)
            orders, conversions, traderData = self.trader.run(state)
            products = group['product'].tolist()
            sandboxLog = ""
            trades_at_timestamp = trade_history_dict.get(timestamp, [])

            for product in products:
                new_trades = []
                for order in orders.get(product, []):
                    trades_done, sandboxLog = self._execute_order(timestamp, order, order_depths_matching, self.current_position, self.cash, trade_history_dict, sandboxLog)
                    new_trades.extend(trades_done)
                if len(new_trades) > 0:
                    own_trades[product] = new_trades
            self.sandbox_logs.append({"sandboxLog": sandboxLog, "lambdaLog": "", "timestamp": timestamp})

            trades_at_timestamp = trade_history_dict.get(timestamp, [])
            if trades_at_timestamp:
                for trade in trades_at_timestamp:
                    product = trade.symbol
                    market_trades[product].append(trade)
            else: 
                for product in products:
                    market_trades[product] = []

            
            for product in products:
                self._mark_pnl(self.cash, self.current_position, order_depths_pnl, self.pnl, product)
                self.pnl_history.append(self.pnl[product])
            self._add_trades(own_trades, market_trades)
        return self._log_trades(self.file_name)
    
    
    def _log_trades(self, filename: str = None):
        if filename is None:
            return 

        self.market_data['profit_and_loss'] = self.pnl_history

        output = ""
        output += "Sandbox logs:\n"
        for i in self.sandbox_logs:
            output += json.dumps(i, indent=2) + "\n"

        output += "\n\n\n\nActivities log:\n"
        market_data_csv = self.market_data.to_csv(index=False, sep=";")
        market_data_csv = market_data_csv.replace("\r\n", "\n")
        output += market_data_csv

        output += "\n\n\n\nTrade History:\n"
        output += json.dumps(self.trades, indent=2)

        with open(filename, 'w+') as file:
            file.write(output)

            
    def _add_trades(self, own_trades: Dict[str, List[Trade]], market_trades: Dict[str, List[Trade]]):
        products = set(own_trades.keys()) | set(market_trades.keys())
        for product in products:
            self.trades.extend([self._trade_to_dict(trade) for trade in own_trades.get(product, [])])
        for product in products:
            self.trades.extend([self._trade_to_dict(trade) for trade in market_trades.get(product, [])])

    def _trade_to_dict(self, trade: Trade) -> dict[str, Any]:
        return {
            "timestamp": trade.timestamp,
            "buyer": trade.buyer,
            "seller": trade.seller,
            "symbol": trade.symbol,
            "currency": "SEASHELLS",
            "price": trade.price,
            "quantity": trade.quantity,
        }
        
    def _construct_trading_state(self, traderData, timestamp, listings, order_depths, 
                                 own_trades, market_trades, position, observations):
        state = TradingState(traderData, timestamp, listings, order_depths, 
                             own_trades, market_trades, position, observations)
        return state
    
        
    def _construct_order_depths(self, group):
        order_depths = {}
        for idx, row in group.iterrows():
            product = row['product']
            order_depth = OrderDepth()
            for i in range(1, 4):
                if f'bid_price_{i}' in row and f'bid_volume_{i}' in row:
                    bid_price = row[f'bid_price_{i}']
                    bid_volume = row[f'bid_volume_{i}']
                    if not pd.isna(bid_price) and not pd.isna(bid_volume):
                        order_depth.buy_orders[int(bid_price)] = int(bid_volume)
                if f'ask_price_{i}' in row and f'ask_volume_{i}' in row:
                    ask_price = row[f'ask_price_{i}']
                    ask_volume = row[f'ask_volume_{i}']
                    if not pd.isna(ask_price) and not pd.isna(ask_volume):
                        order_depth.sell_orders[int(ask_price)] = -int(ask_volume)
            order_depths[product] = order_depth
        return order_depths
    
        
        
    def _execute_buy_order(self, timestamp, order, order_depths, position, cash, trade_history_dict, sandboxLog):
        trades = []
        order_depth = order_depths[order.symbol]

        for price, volume in list(order_depth.sell_orders.items()):
            if price > order.price or order.quantity == 0:
                break

            trade_volume = min(abs(order.quantity), abs(volume))
            if abs(trade_volume + position[order.symbol]) <= int(self.position_limit[order.symbol]):
                trades.append(Trade(order.symbol, price, trade_volume, "SUBMISSION", "", timestamp))
                position[order.symbol] += trade_volume
                self.cash[order.symbol] -= price * trade_volume
                order_depth.sell_orders[price] += trade_volume
                order.quantity -= trade_volume
            else:
                sandboxLog += f"\nOrders for product {order.symbol} exceeded limit of {self.position_limit[order.symbol]} set"
            

            if order_depth.sell_orders[price] == 0:
                del order_depth.sell_orders[price]
        
        trades_at_timestamp = trade_history_dict.get(timestamp, [])
        new_trades_at_timestamp = []
        for trade in trades_at_timestamp:
            if trade.symbol == order.symbol:
                if trade.price < order.price:
                    trade_volume = min(abs(order.quantity), abs(trade.quantity))
                    trades.append(Trade(order.symbol, order.price, trade_volume, "SUBMISSION", "", timestamp))
                    order.quantity -= trade_volume
                    position[order.symbol] += trade_volume
                    self.cash[order.symbol] -= order.price * trade_volume
                    if trade_volume == abs(trade.quantity):
                        continue
                    else:
                        new_quantity = trade.quantity - trade_volume
                        new_trades_at_timestamp.append(Trade(order.symbol, order.price, new_quantity, "", "", timestamp))
                        continue
            new_trades_at_timestamp.append(trade)  

        if len(new_trades_at_timestamp) > 0:
            trade_history_dict[timestamp] = new_trades_at_timestamp

        return trades, sandboxLog
        
        
        
    def _execute_sell_order(self, timestamp, order, order_depths, position, cash, trade_history_dict, sandboxLog):
        trades = []
        order_depth = order_depths[order.symbol]
        
        for price, volume in sorted(order_depth.buy_orders.items(), reverse=True):
            if price < order.price or order.quantity == 0:
                break

            trade_volume = min(abs(order.quantity), abs(volume))
            if abs(position[order.symbol] - trade_volume) <= int(self.position_limit[order.symbol]):
                trades.append(Trade(order.symbol, price, trade_volume, "", "SUBMISSION", timestamp))
                position[order.symbol] -= trade_volume
                self.cash[order.symbol] += price * abs(trade_volume)
                order_depth.buy_orders[price] -= abs(trade_volume)
                order.quantity += trade_volume
            else:
                sandboxLog += f"\nOrders for product {order.symbol} exceeded limit of {self.position_limit[order.symbol]} set"

            if order_depth.buy_orders[price] == 0:
                del order_depth.buy_orders[price]

        trades_at_timestamp = trade_history_dict.get(timestamp, [])
        new_trades_at_timestamp = []
        for trade in trades_at_timestamp:
            if trade.symbol == order.symbol:
                if trade.price > order.price:
                    trade_volume = min(abs(order.quantity), abs(trade.quantity))
                    trades.append(Trade(order.symbol, order.price, trade_volume, "", "SUBMISSION", timestamp))
                    order.quantity += trade_volume
                    position[order.symbol] -= trade_volume
                    self.cash[order.symbol] += order.price * trade_volume
                    if trade_volume == abs(trade.quantity):
                        continue
                    else:
                        new_quantity = trade.quantity - trade_volume
                        new_trades_at_timestamp.append(Trade(order.symbol, order.price, new_quantity, "", "", timestamp))
                        continue
            new_trades_at_timestamp.append(trade)  

        if len(new_trades_at_timestamp) > 0:
            trade_history_dict[timestamp] = new_trades_at_timestamp
                
        return trades, sandboxLog
        
        
        
    def _execute_order(self, timestamp, order, order_depths, position, cash, trades_at_timestamp, sandboxLog):
        if order.quantity == 0:
            return []
        
        order_depth = order_depths[order.symbol]
        if order.quantity > 0:
            return self._execute_buy_order(timestamp, order, order_depths, position, cash, trades_at_timestamp, sandboxLog)
        else:
            return self._execute_sell_order(timestamp, order, order_depths, position, cash, trades_at_timestamp, sandboxLog)
    
    def _mark_pnl(self, cash, position, order_depths, pnl, product):
        order_depth = order_depths[product]
        
        best_ask = min(order_depth.sell_orders.keys())
        best_bid = max(order_depth.buy_orders.keys())
        mid = (best_ask + best_bid)/2
        fair = mid
        if product in self.fair_marks:
            get_fair = self.fair_marks[product]
            fair = get_fair(order_depth)
        
        pnl[product] = cash[product] + fair * position[product]
        


# trader

In [29]:
from datamodel import OrderDepth, UserId, TradingState, Order
from datamodel import Listing, Observation, Order, OrderDepth, ProsperityEncoder, Symbol, Trade, TradingState

from typing import List
import string
import jsonpickle
import numpy as np
import math


class Product:
    RAINFOREST_RESIN = "RAINFOREST_RESIN"
    KELP = "KELP"
    SQUID_INK = "SQUID_INK"

# Updated parameters for three products:
PARAMS = {
    Product.RAINFOREST_RESIN: {
        "fair_value": 10000,
        "take_width": 0.5,          
        "clear_width": 0,
        "disregard_edge": 0.5,
        "join_edge": 1,
        "default_edge": 2,
        "soft_position_limit": 50,
    },
    Product.KELP: {
        "fair_value": 2030,
        "take_width": 0.3,
        "clear_width": 3,
        "prevent_adverse": True,
        "adverse_volume": 15,
        "reversion_beta": -0.4,
        "disregard_edge": 0.5,
        "join_edge": 0.5,
        "default_edge": 0.5,
        "soft_position_limit": 50,
    },
    Product.SQUID_INK: {
        "fair_value": 2000,         # Baseline fair value (starting point)
        "take_width": 0.5,          
        "clear_width": 1,           
        "prevent_adverse": False,
        "adverse_volume": 20,
        "reversion_beta": -0.25,    # Used in mean reverting mode
        "breakout_threshold": 50,   # If midprice deviates more than 30 units, follow the trend
        "disregard_edge": 0.5,
        "join_edge": 0.5,
        "default_edge": 1,
        "soft_position_limit": 50,
    },
}

# backtest run

In [5]:
from datamodel import Listing, Observation
import pandas as pd
import json
import io

In [6]:
def _process_data_(file):
    with open(file, 'r') as file:
        log_content = file.read()
    sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
    sandbox_log =  sections[0].strip()
    activities_log = sections[1].split('Trade History:')[0]
    # sandbox_log_list = [json.loads(line) for line in sandbox_log.split('\n')]
    trade_history =  json.loads(sections[1].split('Trade History:')[1])
    # sandbox_log_df = pd.DataFrame(sandbox_log_list)
    market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
    trade_history_df = pd.json_normalize(trade_history)
    return market_data_df, trade_history_df

### setup

In [7]:
def calculate_kelp_fair(order_depth):
    # assumes order_depth has orders in it 
    best_ask = min(order_depth.sell_orders.keys())
    best_bid = max(order_depth.buy_orders.keys())
    filtered_ask = [price for price in order_depth.sell_orders.keys() if abs(order_depth.sell_orders[price]) >= 15]
    filtered_bid = [price for price in order_depth.buy_orders.keys() if abs(order_depth.buy_orders[price]) >= 15]
    mm_ask = min(filtered_ask) if len(filtered_ask) > 0 else best_ask
    mm_bid = max(filtered_bid) if len(filtered_bid) > 0 else best_bid

    mmmid_price = (mm_ask + mm_bid) / 2
    return mmmid_price

def calculate_squid_ink_fair(order_depth):
    # assumes order_depth has orders in it 
    best_ask = min(order_depth.sell_orders.keys())
    best_bid = max(order_depth.buy_orders.keys())
    filtered_ask = [price for price in order_depth.sell_orders.keys() if abs(order_depth.sell_orders[price]) >= 15]
    filtered_bid = [price for price in order_depth.buy_orders.keys() if abs(order_depth.buy_orders[price]) >= 15]
    mm_ask = min(filtered_ask) if len(filtered_ask) > 0 else best_ask
    mm_bid = max(filtered_bid) if len(filtered_bid) > 0 else best_bid

    mmmid_price = (mm_ask + mm_bid) / 2
    return mmmid_price
    
def calculate_rainforest_resin_fair(order_depth):
    return 10000

In [8]:
listings = {
    'RAINFOREST_RESIN': Listing(symbol='RAINFOREST_RESIN', product='RAINFOREST_RESIN', denomination='SEASHELLS'),
    'KELP': Listing(symbol='KELP', product='KELP', denomination='SEASHELLS'),
    'SQUID_INK': Listing(symbol='SQUID_INK', product='SQUID_INK', denomination='SEASHELLS')
}

position_limit = {
    'RAINFOREST_RESIN': 10,
    'KELP': 10,
    'SQUID_INK':10
}

fair_calculations = {
    "RAINFOREST_RESIN": calculate_rainforest_resin_fair,
    "KELP": calculate_kelp_fair,
    "SQUID_INK": calculate_squid_ink_fair
}


In [43]:
from init_test import Trader
from fixed_logic import Trader as FixedLogicTrader

In [26]:
# market_data, trade_history = _process_data_('round_1_clean_data.log')
market_data, trade_history = _process_data_('log_data\init.log')

# trader = Trader(flush=False)
trader = FixedLogicTrader(params=PARAMS, flush=False)





# backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, "backtests/yuh")
backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history)
backtester.run()
print(backtester.pnl)

{'RAINFOREST_RESIN': 1132, 'KELP': 132.0, 'SQUID_INK': -1649.0}


In [12]:
# market_data, trade_history = _process_data_('round_1_clean_data.log')
market_data, trade_history = _process_data_('log_data\init.log')

trader = Trader(flush=False)

# backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, "backtests/yuh")
backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history)
backtester.run()
print(backtester.pnl)

{'RAINFOREST_RESIN': 700, 'KELP': 132.0, 'SQUID_INK': -1649.0}


In [20]:
# with fair prediction
for day in [-2, -1, 0]:
    market_data = pd.read_csv(f"./data/prices_round_1_day_{day}.csv", sep=";", header=0)
    trade_history = pd.read_csv(f"./data/trades_round_1_day_{day}.csv", sep=";", header=0)

    trader = Trader(flush=False)
    backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, "backtests/volume.log")
    backtester.run()
    print(backtester.pnl)


{'RAINFOREST_RESIN': 11404, 'KELP': 3617.5, 'SQUID_INK': -975.5}
{'RAINFOREST_RESIN': 11495, 'KELP': 3070.0, 'SQUID_INK': -1495.5}
{'RAINFOREST_RESIN': 11083, 'KELP': 2650.0, 'SQUID_INK': -2828.0}


In [ ]:
# day = 0
# market_data = pd.read_csv(f"./data/prices_round_1_day_{day}.csv", sep=";", header=0)
# trade_history = pd.read_csv(f"./data/trades_round_1_day_{day}.csv", sep=";", header=0)

# trader = Trader()
# backtester = Backtester(trader, listings, position_limit, market_data, trade_history, "trade_history_sim.log")
# backtester.run()
# print(backtester.pnl)

# backtest gridsearch

In [36]:
import itertools

def generate_param_combinations(param_grid):
    param_names = param_grid.keys()
    param_values = param_grid.values()
    combinations = list(itertools.product(*param_values))
    return [dict(zip(param_names, combination)) for combination in combinations]

In [37]:
import os
from tqdm import tqdm

def run_backtests(trader, listings, position_limit, fair_calcs, market_data, trade_history, backtest_dir, param_grid, symbol):
    if not os.path.exists(backtest_dir):
        os.makedirs(backtest_dir)

    param_combinations = generate_param_combinations(param_grid[symbol])

    results = []
    for params in tqdm(param_combinations, desc=f"Running backtests for {symbol}", unit="backtest"):
        # trader.params = {symbol: params}
        trader.params[symbol] = params
        backtester = Backtester(trader, listings, position_limit, fair_calcs, market_data, trade_history)
        backtester.run()

        param_str = "-".join([f"{key}={value}" for key, value in params.items()])
        log_filename = f"{backtest_dir}/{symbol}_{param_str}.log"
        # backtester._log_trades(log_filename) # uncomment to save logs # problem with squid_ink

        results.append((params, backtester.pnl[symbol]))

    return results

### setup

In [38]:
listings = {
    'RAINFOREST_RESIN': Listing(symbol='RAINFOREST_RESIN', product='RAINFOREST_RESIN', denomination='SEASHELLS'),
    'KELP': Listing(symbol='KELP', product='KELP', denomination='SEASHELLS'),
    'SQUID_INK': Listing(symbol='SQUID_INK', product='SQUID_INK', denomination='SEASHELLS')
}

position_limit = {
    'RAINFOREST_RESIN': 50,
    'KELP': 50,
    'SQUID_INK':50
}

fair_calculations = {
    "RAINFOREST_RESIN": calculate_rainforest_resin_fair,
    "KELP": calculate_kelp_fair,
    "SQUID_INK": calculate_squid_ink_fair
}



In [39]:
day = 0
market_data = pd.read_csv(f"./data/prices_round_1_day_{day}.csv", sep=";", header=0)
trade_history = pd.read_csv(f"./data/trades_round_1_day_{day}.csv", sep=";", header=0)


### run

In [74]:
# backtest_dir = "backtest_test_clear_width"
backtest_dir = "backtest_test"

param_grid = {
    Product.RAINFOREST_RESIN: {
        "fair_value": [10000],
        "take_width": [1],
        "clear_width": [1], 
        "volume_limit": [0],
        # for making
        "disregard_edge": [2],  # disregards orders for joining or pennying within this value from fair
        "join_edge": [2],# joins orders within this edge 
        "default_edge": [4, 5, 6],
        "soft_position_limit": [9]
    },
    Product.KELP: {
        "fair_value": [1000, 2000, 3000],
        "take_width": [1],
        "clear_width": [0, -0.25],
        "prevent_adverse": [True],
        "adverse_volume": [15],
        "reversion_beta": [-0.229],
        # for making
        "disregard_edge": [1],
        "join_edge": [3],
        "default_edge": [5],
        "soft_position_limit": [50]
    },

    Product.SQUID_INK: {
        "fair_value": [2000],         # Baseline fair value (starting point)
        "take_width": [0.5],          
        "clear_width": [1],           
        "prevent_adverse": [False],
        "adverse_volume": [20],
        "reversion_beta": [-0.25],    # Used in mean reverting mode
        "breakout_threshold": [50],   # If midprice deviates more than 30 units, follow the trend
        "disregard_edge": [0.5],
        "join_edge": [0.5],
        "default_edge": [1],
        "soft_position_limit": [50],
    },
}

trader = Trader(flush=False)

In [75]:
rainforest_resin_results = run_backtests(trader, listings, position_limit, fair_calculations,
                                          market_data, trade_history, backtest_dir, param_grid,
                                            "RAINFOREST_RESIN")
print("RAINFOREST_RESIN results:")
for params, pnl in rainforest_resin_results: 
    print(params)
    print(f"pnl: {pnl}")
    print("="*80)


Running backtests for RAINFOREST_RESIN: 100%|██████████| 3/3 [00:36<00:00, 12.32s/backtest]

RAINFOREST_RESIN results:
{'fair_value': 10000, 'take_width': 1, 'clear_width': 1, 'volume_limit': 0, 'disregard_edge': 2, 'join_edge': 2, 'default_edge': 4, 'soft_position_limit': 9}
pnl: 13642
{'fair_value': 10000, 'take_width': 1, 'clear_width': 1, 'volume_limit': 0, 'disregard_edge': 2, 'join_edge': 2, 'default_edge': 5, 'soft_position_limit': 9}
pnl: 13642
{'fair_value': 10000, 'take_width': 1, 'clear_width': 1, 'volume_limit': 0, 'disregard_edge': 2, 'join_edge': 2, 'default_edge': 6, 'soft_position_limit': 9}
pnl: 13642


In [76]:
max_pnl = 0
for params, pnl in rainforest_resin_results:
    if pnl > max_pnl:
        max_pnl = pnl
        best_params = params

print("Best params for RAINFOREST_RESIN:")
print(best_params)
print(f"Max PnL: {max_pnl}")

Best params for RAINFOREST_RESIN:
{'fair_value': 10000, 'take_width': 1, 'clear_width': 1, 'volume_limit': 0, 'disregard_edge': 2, 'join_edge': 2, 'default_edge': 4, 'soft_position_limit': 9}
Max PnL: 13642


In [70]:
params = PARAMS
params["RAINFOREST_RESIN"].update(best_params)

In [73]:
trader = Trader(params=params, flush=False)
# with fair prediction
for day in [-2, -1, 0]:
    market_data = pd.read_csv(f"./data/prices_round_1_day_{day}.csv", sep=";", header=0)
    trade_history = pd.read_csv(f"./data/trades_round_1_day_{day}.csv", sep=";", header=0)
    backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, "backtests/volume.log")
    backtester.run()
    print(backtester.pnl)


{'RAINFOREST_RESIN': 14924, 'KELP': 5155.0, 'SQUID_INK': 12.0}
{'RAINFOREST_RESIN': 15004, 'KELP': 4627.5, 'SQUID_INK': -5238.5}
{'RAINFOREST_RESIN': 14266, 'KELP': 3845.5, 'SQUID_INK': -2428.0}


In [49]:
symbol = Product.KELP
param_combinations = generate_param_combinations(param_grid[symbol])
results = []

for params in tqdm(param_combinations, desc=f"Running backtests for {symbol}", unit="backtest"):
    # print(params)
    backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history)
    backtester.run()
    param_str = "-".join([f"{key}={value}" for key, value in params.items()])
    if symbol == Product.SQUID_INK:
        log_filename = f"{backtest_dir}/kelp_{param_str}.log"    
    else:
        log_filename = f"{backtest_dir}/{symbol}_{param_str}.log"
    # print(log_filename)
    backtester._log_trades(log_filename)

    results.append((params, backtester.pnl[symbol]))


Running backtests for KELP: 100%|██████████| 6/6 [01:04<00:00, 10.74s/backtest]


In [ ]:
trader = Trader()
kelp_results = run_backtests(trader, listings, position_limit, fair_calculations, market_data, trade_history, backtest_dir, param_grid, "KELP")
print("KELP results:")
for params, pnl in kelp_results: 
    print(params)
    print(f"pnl: {pnl}")
    print("="*80)


Running backtests for KELP: 100%|██████████| 6/6 [00:59<00:00,  9.93s/backtest]

KELP results:
{'fair_value': 1000, 'take_width': 1, 'clear_width': 0, 'prevent_adverse': True, 'adverse_volume': 15, 'reversion_beta': -0.229, 'disregard_edge': 1, 'join_edge': 3, 'default_edge': 5, 'soft_position_limit': 50}
pnl: 880.0
{'fair_value': 1000, 'take_width': 1, 'clear_width': -0.25, 'prevent_adverse': True, 'adverse_volume': 15, 'reversion_beta': -0.229, 'disregard_edge': 1, 'join_edge': 3, 'default_edge': 5, 'soft_position_limit': 50}
pnl: 838.0
{'fair_value': 2000, 'take_width': 1, 'clear_width': 0, 'prevent_adverse': True, 'adverse_volume': 15, 'reversion_beta': -0.229, 'disregard_edge': 1, 'join_edge': 3, 'default_edge': 5, 'soft_position_limit': 50}
pnl: 880.0
{'fair_value': 2000, 'take_width': 1, 'clear_width': -0.25, 'prevent_adverse': True, 'adverse_volume': 15, 'reversion_beta': -0.229, 'disregard_edge': 1, 'join_edge': 3, 'default_edge': 5, 'soft_position_limit': 50}
pnl: 838.0
{'fair_value': 3000, 'take_width': 1, 'clear_width': 0, 'prevent_adverse': True, 'adve

In [ ]:
trader = Trader()
squid_ink_results = run_backtests(trader, listings, position_limit, fair_calculations, market_data, trade_history, backtest_dir, param_grid, "SQUID_INK")
print("SQUID_INK results:")
for params, pnl in squid_ink_results: 
    print(params)
    print(f"pnl: {pnl}")
    print("="*80)

Running backtests for SQUID_INK: 100%|██████████| 1/1 [00:09<00:00,  9.96s/backtest]

SQUID_INK results:
{'fair_value': 2000, 'take_width': 0.5, 'clear_width': 1, 'prevent_adverse': False, 'adverse_volume': 20, 'reversion_beta': -0.25, 'breakout_threshold': 50, 'disregard_edge': 0.5, 'join_edge': 0.5, 'default_edge': 1, 'soft_position_limit': 50}
pnl: 1527.0


In [ ]:
from fixed_logic import Trader as fixed_logic_trader

## analyze

In [ ]:

def analyze_log_files(backtest_dir):
    log_files = [f for f in os.listdir(backtest_dir) if f.endswith('.log')]
    
    results = []
    for log_file in log_files:
        file_path = os.path.join(backtest_dir, log_file)
        
        # Extract symbol and parameters from the file name
        file_name = os.path.splitext(log_file)[0]
        print(file_name)
        symbol, params_str = file_name.split('-', 1)
        params = dict(param.split('=') for param in params_str.split('-'))
        
        # Read the contents of the log file
        with open(file_path, 'r') as file:
            log_content = file.read()
        
        # Store the symbol, parameters, and log content in the results
        results.append({
            'symbol': symbol,
            'params': params,
            'log_content': log_content
        })
    
    return results

# Analyze the log files
log_analysis_results = analyze_log_files(backtest_dir)

# Print the results
for result in log_analysis_results:
    print(f"Symbol: {result['symbol']}")
    print(f"Parameters: {result['params']}")
#     print(f"Log Content:\n{result['log_content']}\n")

KELP_take_width=1-clear_width=-0.25-prevent_adverse=True-adverse_volume=15-reversion_beta=-0.229-disregard_edge=1-join_edge=3-default_edge=5-soft_position_limit=50


ValueError: dictionary update sequence element #1 has length 1; 2 is required

In [ ]:
sorted_starfruit_results = sorted(starfruit_results, key=lambda x: x[1], reverse=True)


In [ ]:
sorted_starfruit_results[0:100]